<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP_7_BERT_Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objective
1. Explore BERT model
2. Perfom spam classifaction with bert-large-uncased model
3. Perform spam classification with Naive Bayes, Logistic Regression, SVM
4. Compare performance of each

In [1]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 6.4 MB/s 
     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 438 kB 42.8 MB/s 
     |████████████████████████████████| 5.9 MB 42.6 MB/s 
     |████████████████████████████████| 1.7 MB 40.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Prepare Dataset

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("/content/drive/MyDrive/Projects/Data/spam.csv", encoding = 'latin-1')
df = df[['v1', 'v2']]
df.columns = ['Label', 'text']
df.head()

Label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [5]:
df["Label"] = df["Label"].astype('category')
df["spam"] = df["Label"].cat.codes
df.head()

Label                                               text  spam
0   ham  Go until jurong point, crazy.. Available only ...     0
1   ham                      Ok lar... Joking wif u oni...     0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...     1
3   ham  U dun say so early hor... U c already then say...     0
4   ham  Nah I don't think he goes to usf, he lives aro...     0

In [10]:
df_spam = df[df['Label']=='spam']
df_spam.shape

(747, 3)

In [11]:
df_ham = df[df['Label']=='ham']
df_ham.shape

(4825, 3)

In [13]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 3)

In [14]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 3)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['spam'], stratify=df_balanced['spam'])

In [16]:
X_train.head(4)

2216    OK i'm waliking ard now... Do u wan me 2 buy a...
5467    Get your garden ready for summer with a FREE s...
1608    Are your freezing ? Are you home yet ? Will yo...
3718    Thanks for your ringtone order, reference numb...
Name: text, dtype: object

#Import BERT

In [8]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4")

#Build Model

In [9]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [17]:
len(X_train)

1120

#Train the model

In [18]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 534s 15s/step - loss: 0.6746 - accuracy: 0.5786 - precision: 0.5791 - recall: 0.5750
Epoch 2/10
35/35 [==============================] - 515s 15s/step - loss: 0.5872 - accuracy: 0.7063 - precision: 0.7151 - recall: 0.6857
Epoch 3/10
35/35 [==============================] - 519s 15s/step - loss: 0.5397 - accuracy: 0.7625 - precision: 0.7634 - recall: 0.7607
Epoch 4/10
35/35 [==============================] - 522s 15s/step - loss: 0.4919 - accuracy: 0.8170 - precision: 0.8187 - recall: 0.8143
Epoch 5/10
20/35 [================>.............] - ETA: 3:44 - loss: 0.4628 - accuracy: 0.8484 - precision: 0.8270 - recall: 0.9027